In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k', prompt=False)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/gw/.surprise_data/ml-100k


In [5]:
algo = SVD()
algo.fit(trainset)

In [6]:
predictions = algo.test(testset)
print(type(predictions))
print(len(predictions))
print(predictions[:5])

<class 'list'>
25000
[Prediction(uid='120', iid='282', r_ui=4.0, est=3.7474195757677067, details={'was_impossible': False}), Prediction(uid='882', iid='291', r_ui=4.0, est=3.5199816982844303, details={'was_impossible': False}), Prediction(uid='535', iid='507', r_ui=5.0, est=4.095071183115341, details={'was_impossible': False}), Prediction(uid='697', iid='244', r_ui=5.0, est=3.7737860006705883, details={'was_impossible': False}), Prediction(uid='751', iid='385', r_ui=4.0, est=3.7260019496235772, details={'was_impossible': False})]


In [10]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.7474195757677067),
 ('882', '291', 3.5199816982844303),
 ('535', '507', 4.095071183115341)]

In [11]:
accuracy.rmse(predictions)

RMSE: 0.9471


0.9470800487454554

In [12]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('../../../datasets/movielens/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [13]:
from surprise.model_selection import cross_validate

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8714  0.8679  0.8730  0.8724  0.8761  0.8722  0.0026  
MAE (testset)     0.6709  0.6672  0.6692  0.6698  0.6722  0.6699  0.0017  
Fit time          0.67    0.66    0.63    0.62    0.64    0.64    0.02    
Test time         0.09    0.08    0.17    0.07    0.07    0.09    0.04    


{'test_rmse': array([0.87140953, 0.86792468, 0.87304525, 0.87237074, 0.87609231]),
 'test_mae': array([0.67088479, 0.66719535, 0.66921508, 0.66975672, 0.67223949]),
 'fit_time': (0.6690247058868408,
  0.662431001663208,
  0.629669189453125,
  0.6233625411987305,
  0.6402714252471924),
 'test_time': (0.08678364753723145,
  0.07537031173706055,
  0.16559767723083496,
  0.07381677627563477,
  0.07091784477233887)}

In [14]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs':[20, 40, 60], 'n_factors':[50, 100, 200]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8770150022608613
{'n_epochs': 20, 'n_factors': 50}


In [16]:
from surprise.dataset import DatasetAutoFolds

ratings = pd.read_csv('../../../datasets/movielens/ratings.csv')
ratings.to_csv('../../../datasets/movielens/ratings_noh.csv', index=False, header=False)

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='../../../datasets/movielens/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [17]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [19]:
movies = pd.read_csv('../../../datasets/movielens/movies.csv')

movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('none')

print(movies[movies['movieId'] == 42])

none
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [20]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [21]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    total_movies = movies['movieId'].tolist()

    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies))
    print('추천 대상 영화 수:', len(unseen_movies))
    print('전체 영화 수:', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46
추천 대상 영화 수: 9696
전체 영화 수: 9742


In [23]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_ratings = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_ratings)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

for top_movie in top_movie_preds:
    print(top_movie[1], top_movie[2])

평점 매긴 영화 수: 46
추천 대상 영화 수: 9696
전체 영화 수: 9742
Usual Suspects, The (1995) 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) 4.281663842987387
Pulp Fiction (1994) 4.278152632122759
Silence of the Lambs, The (1991) 4.226073566460876
Godfather, The (1972) 4.1918097904381995
Streetcar Named Desire, A (1951) 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) 4.108009609093436
Goodfellas (1990) 4.083464936588478
Glory (1989) 4.07887165526957
